# Create molecular networks
This notebooks creates a graphml file for the case study data to enable visualizing mol networks in cytoscape

### Load MS2Deepscore model

In [1]:
from ms2deepscore.models import load_model

filename_model = "../../../data/pytorch/new_corinna_included/trained_models/both_mode_ionmode_precursor_mz_2000_2000_2000_layers_500_embedding_2024_10_01_20_08_53/ms2deepscore_model.pt"
model = load_model(filename_model)

c:\users\jonge094\pycharmprojects\ms2deepscore\ms2deepscore\ms2deepscore\models\load_model.py:34: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model_settings = torch.load(f

The model version (2.0.0) does not match the version of MS2Deepscore (2.1.0), consider downloading a new model or changing the MS2Deepscore version


### Create spectral similarity scores
The spectrum file "./cleaned_spectra_pos_neg_with_numbering.mgf" was created in pre_processing_spectra

In [2]:
from matchms.Pipeline import Pipeline, create_workflow
from ms2deepscore import MS2DeepScore

workflow = create_workflow(
    query_filters=[],
    score_computations=[
        [MS2DeepScore, {"model": model}],
        ],
)
pipeline = Pipeline(workflow)
report = pipeline.run( "../../../data/data_elena/cleaned_spectra_pos_neg_with_numbering.mgf")

2024-10-11 08:57:31,283:WARNING:matchms:Pipeline:No logging file was defined.Logging messages will not be written to file.
2024-10-11 08:57:31,287:WARNING:matchms:SpectrumProcessor:No filters have been specified, so spectra were not filtered


Processing spectra: 2909it [00:04, 710.25it/s]
2909it [00:32, 88.85it/s]


### Create a network
The pipeline.scores contain all the scores. To make a molecular network only some of the similarity scores are stored. They are only stored if the score is at least 0.85 and each node (spectrum) is only connected to the top 5 highest similarity scores and only if it is in the tup 5 of that other spectrum as well. 

This is the common approach for creating molecular networks (exact settings vary) and allows for getting visually pleasing mol networks (preventing giant hairbals).

In [3]:
from matchms.networking import SimilarityNetwork

# Define settings
ms2ds_network = SimilarityNetwork(
    identifier_key="query_spectrum_nr",
    score_cutoff=0.90,  # higher numbers produce more isolated sub-graphs
    max_links=5 ,  # lower number makes sparser networks
    link_method="mutual",  # mutual means: link is only added if in top list of both nodes
)

# Compute the graph (takes some time)
ms2ds_network.create_network(pipeline.scores, score_name="MS2DeepScore")

### save to graphml

In [4]:
# Export to graphml
ms2ds_network.export_to_graphml("ms2ds_graph.graphml")


### Load into cytoscape

The graphml file can be loaded into cytoscape: https://cytoscape.org/ This is an open source platform for visualizing graphs. 


To recreate the case study results:
- Open cytoscape
- Load in the above created graphml file.
- Load in ms2query annotations as table (see file add_annotations.ipynb)
- Set style settings (or load in a style file)
- Set up chemviz to visualize chemical information.
- Explore your data!